In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from spot.utils import cst, read_file, write_file, SpecialNames
import os

if os.getcwd().endswith('scripts'):
    os.chdir('..')

parsed = cst.parse_module(read_file(f"data/code/bad_code_1.py"))

In [33]:
from spot.type_checking import AnnotCollector, AnnotApplier, MypyChecker

collector = AnnotCollector()
parsed.visit(collector)
collector.annotations.keys()

dict_keys([('fib', '<return>'), ('fib', 'n'), ('t_add', '<return>'), ('t_add', 'x'), ('t_add', 'y'), ('x',)])

In [34]:
# TODO: make unit tests
# print(parsed.visit(AnnotApplier(("fib", SpecialNames.Return), cst.Annotation(cst.Name("int")))).code)
applier = AnnotApplier({
    ("fib", "n"): cst.Annotation(cst.Name("int")),
    ("fib", SpecialNames.Return): cst.Annotation(cst.Name("int")),
})
print(parsed.visit(applier).code)
# applier.prefixes

from typing import List

# A recursive fibonacci function
def fib(n: int) -> int:
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def t_add(x: str, y: str) -> int:
    r = x + y
    return r

x: int = fib(3)


In [35]:
checker = MypyChecker(".venv/bin/dmypy", "data/code")
checker.mypy_version

'dmypy 0.942\n'

In [36]:
check_r = checker.check_file(".")
print(check_r.output_str)
check_r.num_error_dict

chekcing!
bad_code_1.py:6: error: Incompatible return value type (got "int", expected "List[int]")
bad_code_1.py:8: error: Incompatible return value type (got "int", expected "List[int]")
bad_code_1.py:10: error: Unsupported operand types for - ("str" and "int")
bad_code_1.py:10: error: Argument 1 to "fib" has incompatible type "int"; expected "str"
bad_code_1.py:14: error: Incompatible return value type (got "str", expected "int")
bad_code_1.py:16: error: Incompatible types in assignment (expression has type "List[int]", variable has type "int")
bad_code_1.py:16: error: Argument 1 to "fib" has incompatible type "int"; expected "str"
bad_code_2.py:3: error: Argument 1 to "fib" has incompatible type "int"; expected "str"
Found 8 errors in 2 files (checked 3 source files)



{'bad_code_1.py': 7, 'bad_code_2.py': 1}

In [37]:
out_checker = MypyChecker(".venv/bin/dmypy", "data/code_output")

Daemon stopped


In [44]:
write_file("data/code_output/bad_code_1.py", parsed.code)
print("before: ", checker.check_file(".").num_errors)
write_file("data/code_output/bad_code_1.py", parsed.visit(applier).code)
print("after: ", out_checker.check_file(".").num_errors)

chekcing!
before:  8
chekcing!
after:  1
